# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd
import numpy as np

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [2]:
# Define column names
column_names = ['PostalCode', 'Borough', 'Neighborhood']

Getting the data and sorting out values into a jagged array.

In [3]:
# download table
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

# find table in html page
My_table = soup.find('table',{'class':'wikitable sortable'})
MyTableCells = My_table.findAll('td')
# variables for extraction
My_list = []
jagged = []

# counter for list items (every 3 items will be a row)
# starting with 0, reset at 2 
x = 0

    # sort out data
for i in MyTableCells:
    if x == 2:
        My_list.append(i.text.strip('\n'))
        
        # if Borough is not "not assigned", add to the jagged array
        if My_list[1] != 'Not assigned':
            if My_list[2] == 'Not assigned':
                My_list[2] = My_list[1]
            jagged.append(My_list)
        My_list = []
        x = 0
    else:
        My_list.append(i.text)
        x += 1

print('Extraction is complete.')

Extraction is complete.


Making the dataframe workbench.

In [4]:
toronto_df_wb = pd.DataFrame(jagged, columns=column_names)
toronto_df_wb.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


We need to bundle up neighborhoods by PostalCode. For that I'm making a new dataframe.

In [5]:
new_df = pd.DataFrame(toronto_df_wb.groupby('PostalCode')['Neighborhood'].apply(list))
new_df.head()

,Neighborhood
PostalCode,
M1B,"[Rouge, Malvern]"
M1C,"[Highland Creek, Rouge Hill, Port Union]"
M1E,"[Guildwood, Morningside, West Hill]"
M1G,[Woburn]
M1H,[Cedarbrae]


I'm merging our existing DataFrame (toronto_df_wb) with the listed neighborhoods dataframe (new_df).
And then dropping the duplicates (toronto_df_wb has multiple duplicate PostalCode values, therefore we have to remove them duplicates).

In [6]:
dff = pd.merge(toronto_df_wb[['PostalCode','Borough']], new_df[['Neighborhood']], left_on='PostalCode', right_index=True)
toronto_df = dff.loc[dff.astype(str).drop_duplicates().index]
toronto_df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Harbourfront, Regent Park]"
4,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
6,M7A,Queen's Park,[Queen's Park]
7,M9A,Etobicoke,[Islington Avenue]
8,M1B,Scarborough,"[Rouge, Malvern]"
10,M3B,North York,[Don Mills North]
11,M4B,East York,"[Woodbine Gardens, Parkview Hill]"
13,M5B,Downtown Toronto,"[Ryerson, Garden District]"


### We've got a nice and clean dataframe once again.

In [7]:
toronto_df.shape

(103, 3)

Importing the longitude and latitude values into a new dataframe, then renaming the column to fit the other dataframe we previously created.

In [8]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')
df_ll.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_ll.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Mergint the two dataframes into one, so we get the finished product table.

In [9]:
df_toronto_ll = pd.merge(toronto_df, df_ll, on='PostalCode')
df_toronto_ll.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Harbourfront, Regent Park]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]",43.718518,-79.464763
4,M7A,Queen's Park,[Queen's Park],43.662301,-79.389494
5,M9A,Etobicoke,[Islington Avenue],43.667856,-79.532242
6,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
7,M3B,North York,[Don Mills North],43.745906,-79.352188
8,M4B,East York,"[Woodbine Gardens, Parkview Hill]",43.706397,-79.309937
9,M5B,Downtown Toronto,"[Ryerson, Garden District]",43.657162,-79.378937
